In [ ]:
# !pip install tensorflow

In [13]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM  , Dense , Input , Embedding , RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import optimizers

In [14]:
# i upload this file as run time form pc
# se when we we again run then we want to again uplod this file 

with open('fra.txt' , 'r', encoding='utf-8') as file:
  lines = file.readlines()
  words = []
  for line in lines:
    word = line.strip().split('\t')
    # word = word.split('\t')
    words.append(word)

In [15]:
words[:5]

[['Go.',
  'Va !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'],
 ['Go.',
  'Marche.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)'],
 ['Go.',
  'Bouge !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)'],
 ['Hi.',
  'Salut !',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)'],
 ['Hi.',
  'Salut.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)']]

In [16]:
eng_fre = np.array(words)

eng_fre[:5]

array([['Go.', 'Va !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'],
       ['Go.', 'Marche.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)'],
       ['Go.', 'Bouge !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)'],
       ['Hi.', 'Salut !',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)'],
       ['Hi.', 'Salut.',
        'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)']],
      dtype='<U102')

In [17]:
eng_fre.shape

(136926, 3)

In [18]:
# so we see that data set are to long so we just take first 60000 data
eng_fre = eng_fre[:60000]
eng_fre.shape

(60000, 3)

In [19]:
eng_fre = eng_fre[:,[0,1]]

In [20]:
eng_fre[1]

array(['Go.', 'Marche.'], dtype='<U102')

In [21]:
#  Data cleaning 
# remove panctution 
eng_fre[:,0] = [s.translate(str.maketrans("" , "" , string.punctuation))for s in eng_fre[:,0]]
eng_fre[:,1] = [s.translate(str.maketrans("" , "" , string.punctuation))for s in eng_fre[:,1]]

In [22]:
# just try it how punctuation works

str1 = "hello. how's your day. "
str1.translate(str.maketrans("" , "" , string.punctuation))


'hello hows your day '

In [23]:
# now we remove all the . ,' or any punction from the string
# eng_fre

In [24]:
# convert in lowercase 
for i in range(len(eng_fre)):
  eng_fre[i][0] = eng_fre[i][0].lower()
  eng_fre[i][1] = eng_fre[i][1].lower()
eng_fre

array([['go', 'va '],
       ['go', 'marche'],
       ['go', 'bouge '],
       ...,
       ['were all having lunch', 'nous sommes tous en train de déjeuner'],
       ['were all having lunch', 'nous sommes tous en train de dîner'],
       ['were all in agreement', 'nous sommes tous daccord']],
      dtype='<U102')

In [25]:
def tokenization(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# prepare english tokenizer 

eng_tokenizer = tokenization(eng_fre[:,0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = 15
print("English vocabulary size : %d" % eng_vocab_size)
eng_tokenizer

English vocabulary size : 6855


In [26]:
fre_tokenizer = tokenization(eng_fre[:,1])
fre_vocab_size = len(fre_tokenizer.word_index) + 1
fre_length = 15
print("french vocabulary size : %d" % fre_vocab_size)

french vocabulary size : 15831


In [27]:
# encode and pad sequences , padding to a maximumsequence length
def encode_sequence(tokenizer , length , lines):
  # integer encode sequences
  seq = tokenizer.texts_to_sequences(lines)
  # pad sequences with 0 values 
  seq = pad_sequences(seq , maxlen = length , padding = 'post')
  print(seq.shape)
  return seq


  

we encode the english sentances as input and french sentance as target

In [28]:
from sklearn.model_selection import train_test_split

# split into train test set 
train , test = train_test_split(eng_fre , test_size = 0.2 , random_state = 12)

In [29]:
# x = eng_fre[:,0].reshape(9153,1)

In [30]:
# eng = ['hiii' , 'hello' , 'Riya' , 'how are you', 'i am fine ' , 'what are you doing', 'yes , please comming']
# tokenizer = Tokenizer()
# tok = tokenizer.fit_on_texts(eng)
# seq = tokenizer.texts_to_sequences(tok)
# seq

In [31]:
print(train.shape , train[:,0]) #, print(test.shape , test )

(48000, 2) ['we need to fix this' 'do me a favor' 'whats the daily rate' ...
 'the nation was growing' 'tom knew everything' 'i love barbecue']


In [32]:
#  preparing training data
trainX = encode_sequence(eng_tokenizer , eng_length , train[:,0])
trainY = encode_sequence(fre_tokenizer , fre_length , train[:,1])

#  preparing test data
testX = encode_sequence(eng_tokenizer , eng_length , train[:,0])
testY = encode_sequence(fre_tokenizer , fre_length , train[:,1])


(48000, 15)
(48000, 15)
(48000, 15)
(48000, 15)


Define Seq to Seq model

In [33]:
#  in_vocab = input vocab size 
# out_vocab = output vocab size 
# in_timestep = length of sentense at time stems in our case we set it as 15 (eng_length)
# out_timestep = length of sentense at time stems in our case we set it as 15 (fre_length)
# units = number of neurons
def define_model(in_vocab , out_vocab , in_timesteps , out_timesteps , units):
  model = Sequential()
  # next two lines for the encoder 
  model.add(Embedding(in_vocab , units , input_length = in_timesteps))
  model.add(LSTM(units))
  #  the repeatvector is responcible for repeate input   // it repeate the input basis of out_timesteps 
  model.add(RepeatVector(out_timesteps))
  #  this two line decode the input
  model.add(LSTM(units , return_sequences = True))
  model.add(Dense(out_vocab , activation = 'softmax'))
  return model

In [34]:
# we are using the RMSproof optimizer in this model as its usually a good choise when we work with RNN
# model complition 

model = define_model(eng_vocab_size , fre_vocab_size , eng_length , fre_length , 512)
rms = optimizers.RMSprop(lr = 0.001)
model.compile(optimizer = rms , loss = 'sparse_categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [35]:
#  train mode  

history = model.fit(trainX , trainY.reshape(trainY.shape[0] , trainY.shape[1], 1) , epochs = 50 , batch_size = 100 , validation_split = 0.2)

Epoch 1/50
384/384 [==============================] - 26s 47ms/step - loss: 1.9896 - val_loss: 1.7975
Epoch 2/50
384/384 [==============================] - 17s 45ms/step - loss: 1.6964 - val_loss: 1.6425
Epoch 3/50
384/384 [==============================] - 19s 49ms/step - loss: 1.5571 - val_loss: 1.5278
Epoch 4/50
384/384 [==============================] - 18s 47ms/step - loss: 1.4421 - val_loss: 1.4561
Epoch 5/50
384/384 [==============================] - 19s 51ms/step - loss: 1.3526 - val_loss: 1.4035
Epoch 6/50
384/384 [==============================] - 20s 52ms/step - loss: 1.2731 - val_loss: 1.3220
Epoch 7/50
384/384 [==============================] - 19s 50ms/step - loss: 1.1971 - val_loss: 1.2795
Epoch 8/50
384/384 [==============================] - 21s 54ms/step - loss: 1.1281 - val_loss: 1.2440
Epoch 9/50
384/384 [==============================] - 19s 50ms/step - loss: 1.0658 - val_loss: 1.2048
Epoch 10/50
384/384 [==============================] - 19s 50ms/step - loss: 1.007

In [39]:
# preds = model.predict(testX.reshape((testX.shape[0] , testX.shape[1],1)))
preds = model.predict(testX)

In [41]:
# preds.argmax(1)
#  this prediction is in integer sequences to we need to convert it in word 

In [42]:
#  this prediction is in integer sequences to we need to convert it in word 

def get_words(n, tokenizer):
  for word , index in tokenizer.word_index.items():
    if index == n:
      return word
    return None

In [44]:
# convert prediction into sentances 

pred_text = []
for i in preds:
  temp = []
  for j in range(len(i)):
    t = get_words(i[j] , fre_tokenizer)
    if j > 0 :
      if (t == get_words(i[j-1] , fre_tokenizer) or t == None):
        temp.append('')
      else:
        temp.append(t)
    else:
      if(t == None):
        temp.append('')
      else:
        temp.append(t)
  pred_text.append(' '.join(temp))

In [45]:
pred_df = pd.DataFrame({'actual' : test[:0] , 'predicted' : pred_text})

In [46]:
pred_df